# Destination From (A) to (B)

In [2]:
!pip install folium
!pip install flexpolyline

In [9]:
import requests                         #Import the requests library, which is used for making HTTP requests to external APIs
import folium                           # Import the folium library, which allows you to generate interactive maps in Python.
import flexpolyline as fp               # Decoder for HERE API polylines
from IPython.display import display     # Import the display function from IPython.display. This is used to render the interactive map in Jupyter or Google Colab.

# Your HERE API key
here_API_KEY = "sFR8fGF9tv_-oQTgOPCZe4hcUH4kurAUCLc-zRmxcsA"  # Replace with your actual API key

# Coordinates for point A and point B (ensure no spaces after the comma)
start_coords = "24.678499,46.759968"
end_coords = "24.854194,46.712939" #    # Example: Riyadh


# Directions API request URL (adding 'polyline' in the return parameter)
directions_url = f"https://router.hereapi.com/v8/routes?transportMode=car&origin={start_coords}&destination={end_coords}&return=summary,actions,polyline&apiKey={here_API_KEY}"

# Send the request to the HERE Directions API
response = requests.get(directions_url)

# Check if the response was successful
if response.status_code == 200:
    data = response.json()

    # Extract route information
    if 'routes' in data and data['routes']:   # Ensure that the routes data exists in the API response and that it contains route information.
        route = data['routes'][0]             # Select the first route from the available routes provided by the API.
        summary = route['sections'][0]['summary']  # Extract the summary of the route (e.g., total distance and duration) from the first section of the route.
        actions = route['sections'][0].get('actions', [])  # Extract the turn-by-turn directions (actions) from the route, if available.
        polyline_data = route['sections'][0]['polyline']  # Extract the polyline data (an encoded series of latitude/longitude points representing the route) from the route section.

        # Decode the polyline into lat/lon pairs
        route_coords = fp.decode(polyline_data)  # List of (latitude, longitude) pairs

        # Print the summary of the route
        print("\nRoute Summary:")
        print(f"Distance: {summary['length']/1000} km")
        print(f"Duration: {summary['duration']/60} minutes")

        # Process and format the turn-by-turn directions
        if actions:
            print("\nTurn-by-Turn Directions:")
            for action in actions:
                # Create a human-readable instruction based on the action type and direction
                action_type = action.get('action', 'move')
                direction = action.get('direction', '')
                length = action.get('length', 0) / 1000  # Convert length to kilometers

                if action_type == "depart":
                    print(f"Depart and start your journey.")
                elif action_type == "turn" and direction:
                    print(f"Turn {direction}. Continue for {length:.2f} km.")
                elif action_type == "ramp" and direction:
                    print(f"Take the {direction} ramp. Continue for {length:.2f} km.")
                elif action_type == "arrive":
                    print(f"Arrive at your destination.")
                else:
                    print(f"{action_type.capitalize()} for {length:.2f} km.")

        # Create a folium map centered between the start and end points
        mid_lat = (float(start_coords.split(",")[0]) + float(end_coords.split(",")[0])) / 2
        mid_lon = (float(start_coords.split(",")[1]) + float(end_coords.split(",")[1])) / 2
        route_map = folium.Map(location=[mid_lat, mid_lon], zoom_start=14)

        # Add a marker for the starting point (A)
        folium.Marker(
            location=[float(start_coords.split(",")[0]), float(start_coords.split(",")[1])],
            popup="Start",
            icon=folium.Icon(color="green")
        ).add_to(route_map)

        # Add a marker for the ending point (B)
        folium.Marker(
            location=[float(end_coords.split(",")[0]), float(end_coords.split(",")[1])],
            popup="End",
            icon=folium.Icon(color="red")
        ).add_to(route_map)

        # Add the route polyline to the map
        folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.8).add_to(route_map)

        # Display the map in Google Colab or Jupyter
        display(route_map)
else:
    print(f"Error {response.status_code}: {response.text}")



Route Summary:
Distance: 31.046 km
Duration: 42.45 minutes

Turn-by-Turn Directions:
Depart and start your journey.
Turn right. Continue for 0.11 km.
Turn right. Continue for 1.10 km.
Turn left. Continue for 0.09 km.
Keep for 0.10 km.
Turn left. Continue for 1.53 km.
Turn right. Continue for 0.98 km.
Turn right. Continue for 0.98 km.
Continue for 1.00 km.
Continue for 0.08 km.
Keep for 0.06 km.
Continue for 4.51 km.
Keep for 3.52 km.
Keep for 0.88 km.
Keep for 2.90 km.
Keep for 0.48 km.
Keep for 1.18 km.
Continue for 2.71 km.
Take the right ramp. Continue for 2.68 km.
Take the right ramp. Continue for 2.32 km.
Take the right ramp. Continue for 0.24 km.
Roundaboutexit for 1.96 km.
Turn right. Continue for 0.26 km.
Turn right. Continue for 0.30 km.
Turn right. Continue for 0.01 km.
Turn right. Continue for 0.05 km.
Roundaboutexit for 0.11 km.
Roundaboutexit for 0.33 km.
Roundaboutexit for 0.14 km.
Turn left. Continue for 0.01 km.
Turn right. Continue for 0.19 km.
Turn right. Continue fo

# Distination From (A) to (B), Comparing Highest with Least Traffic Situation

In [3]:
import requests
import folium
import flexpolyline as fp  # Decoder for HERE API polylines
from IPython.display import display

# Your HERE API key
here_API_KEY = "sFR8fGF9tv_-oQTgOPCZe4hcUH4kurAUCLc-zRmxcsA"  # Replace with your actual API key

# Coordinates for point A and point B (ensure no spaces after the comma)
start_coords = "24.678499,46.759968"
end_coords = "24.854194,46.712939"  # Example: Riyadh

# Directions API request URL with alternatives
directions_url = f"https://router.hereapi.com/v8/routes?transportMode=car&origin={start_coords}&destination={end_coords}&return=summary,actions,polyline&alternatives=3&apiKey={here_API_KEY}"

# Send the request to the HERE Directions API to get multiple route options
response = requests.get(directions_url)

# Check if the response was successful
if response.status_code == 200:
    data = response.json()

    # Extract the routes
    if 'routes' in data and data['routes']:
        routes = data['routes']  # List of possible routes
        least_congested_route = None
        most_congested_route = None
        least_congestion_score = float('inf')  # Set a high initial value to compare congestion
        most_congestion_score = -float('inf')  # Set a low initial value to compare congestion

        # Loop through each route and calculate its traffic conditions
        for route in routes:
            polyline_data = route['sections'][0]['polyline']  # Encoded polyline data for the route
            route_coords = fp.decode(polyline_data)  # Decode the polyline into lat/lon pairs

            # Calculate the bounding box dynamically for traffic checking
            latitudes = [coord[0] for coord in route_coords]
            longitudes = [coord[1] for coord in route_coords]
            min_lat, max_lat = min(latitudes), max(latitudes)
            min_lng, max_lng = min(longitudes), max(longitudes)

            # Create a bounding box with a small margin to cover the route
            margin = 0.01
            bbox = f"{min_lng - margin},{min_lat - margin},{max_lng + margin},{max_lat + margin}"

            # Step 2: Get the traffic information along this route
            traffic_url = 'https://data.traffic.hereapi.com/v7/flow'
            traffic_params = {
                'apikey': here_API_KEY,
                'in': f"bbox:{bbox}",
                'units': 'metric',
                'locationReferencing': 'shape'
            }

            traffic_response = requests.get(traffic_url, params=traffic_params)

            if traffic_response.status_code == 200:
                traffic_data = traffic_response.json()

                # Calculate traffic congestion score (sum of jam factors along the route)
                congestion_score = 0
                for segment in traffic_data['results']:
                    current_flow = segment['currentFlow']
                    jam_factor = current_flow.get('jamFactor', 0)
                    congestion_score += jam_factor  # Sum up the jam factors for this route

                # Update least congested route if this one is better
                if congestion_score < least_congestion_score:
                    least_congestion_score = congestion_score
                    least_congested_route = route

                # Update most congested route if this one is worse
                if congestion_score > most_congestion_score:
                    most_congestion_score = congestion_score
                    most_congested_route = route

        # After evaluating all routes, display the least and most congested routes
        if least_congested_route and most_congested_route:
            print(f"Least congested route selected with congestion score: {least_congestion_score}")
            print(f"Most congested route selected with congestion score: {most_congestion_score}")

            # Initialize a folium map centered between the start and end points
            mid_lat = (float(start_coords.split(",")[0]) + float(end_coords.split(",")[0])) / 2
            mid_lon = (float(start_coords.split(",")[1]) + float(end_coords.split(",")[1])) / 2
            route_map = folium.Map(location=[mid_lat, mid_lon], zoom_start=14)

            # Add a marker for the starting point (A)
            folium.Marker(
                location=[float(start_coords.split(",")[0]), float(start_coords.split(",")[1])],
                popup="Start",
                icon=folium.Icon(color="green")
            ).add_to(route_map)

            # Add a marker for the ending point (B)
            folium.Marker(
                location=[float(end_coords.split(",")[0]), float(end_coords.split(",")[1])],
                popup="End",
                icon=folium.Icon(color="red")
            ).add_to(route_map)

            # Display the least congested route
            least_polyline_data = least_congested_route['sections'][0]['polyline']
            least_route_coords = fp.decode(least_polyline_data)
            folium.PolyLine(least_route_coords, color="blue", weight=5, opacity=0.8,
                            popup="Least Congested Route").add_to(route_map)

            # Display the most congested route
            most_polyline_data = most_congested_route['sections'][0]['polyline']
            most_route_coords = fp.decode(most_polyline_data)
            folium.PolyLine(most_route_coords, color="red", weight=5, opacity=0.8,
                            popup="Most Congested Route").add_to(route_map)

            # Display the map in Google Colab or Jupyter
            display(route_map)

else:
    print(f"Error {response.status_code}: {response.text}")


Least congested route selected with congestion score: 3905.600000000006
Most congested route selected with congestion score: 5258.199999999993


# Multiple Destinations

In [7]:
import requests
import folium
import flexpolyline as fp  # Decoder for HERE API polylines
from IPython.display import display

# Your HERE API key
here_API_KEY = "sFR8fGF9tv_-oQTgOPCZe4hcUH4kurAUCLc-zRmxcsA"  # Replace with your actual API key

# Coordinates for the start, via points, and end (ensure no spaces after commas)
start_coords = "24.854194,46.712939"  # Start point
via_coords_1 = "24.754636,46.635893"  # First via point
via_coords_2 = "24.738056,46.710167"  # Second via point
end_coords = "24.678499,46.759968"    # End point



# Create the Directions API request URL, including all via points
directions_url = f"https://router.hereapi.com/v8/routes?transportMode=car&origin={start_coords}&destination={end_coords}&via={via_coords_1}&via={via_coords_2}&return=summary,actions,polyline&apiKey={here_API_KEY}"

# Send the request to the HERE Directions API
response = requests.get(directions_url)

# Check if the response was successful
if response.status_code == 200:
    data = response.json()

    # Extract route information
    if 'routes' in data and data['routes']:
        route = data['routes'][0]  # Take the first route
        sections = route['sections']  # Sections of the route (each between waypoints)

        # Initialize a folium map centered between the start and end points
        mid_lat = (float(start_coords.split(",")[0]) + float(end_coords.split(",")[0])) / 2
        mid_lon = (float(start_coords.split(",")[1]) + float(end_coords.split(",")[1])) / 2
        route_map = folium.Map(location=[mid_lat, mid_lon], zoom_start=12)

        # Add markers for the starting point
        folium.Marker(
            location=[float(start_coords.split(",")[0]), float(start_coords.split(",")[1])],
            popup="Start",
            icon=folium.Icon(color="green")
        ).add_to(route_map)

        # Add the via points (step-by-step)
        folium.Marker(
            location=[float(via_coords_1.split(",")[0]), float(via_coords_1.split(",")[1])],
            popup="Via Point 1",
            icon=folium.Icon(color="blue")
        ).add_to(route_map)

        folium.Marker(
            location=[float(via_coords_2.split(",")[0]), float(via_coords_2.split(",")[1])],
            popup="Via Point 2",
            icon=folium.Icon(color="blue")
        ).add_to(route_map)

        # Add a marker for the ending point (End)
        folium.Marker(
            location=[float(end_coords.split(",")[0]), float(end_coords.split(",")[1])],
            popup="End",
            icon=folium.Icon(color="red")
        ).add_to(route_map)

        # Process each section (start -> via -> via -> end)
        for section in sections:
            summary = section['summary']
            polyline_data = section['polyline']  # Encoded polyline data

            # Decode the polyline into lat/lon pairs
            route_coords = fp.decode(polyline_data)

            # Add the route segment polyline to the map
            folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.8).add_to(route_map)

            # Print section summary (optional)
            print(f"\nSection Summary: Distance: {summary['length']/1000:.2f} km, Duration: {summary['duration']/60:.2f} minutes")

        # Display the map in Google Colab or Jupyter
        display(route_map)
else:
    print(f"Error {response.status_code}: {response.text}")



Section Summary: Distance: 19.93 km, Duration: 27.17 minutes

Section Summary: Distance: 11.09 km, Duration: 16.48 minutes

Section Summary: Distance: 13.66 km, Duration: 20.67 minutes


# Going to Multiple Destinations, Based on Least Traffic

In [6]:
import requests
import folium
import flexpolyline as fp  # Decoder for HERE API polylines
from IPython.display import display

# Your HERE API key
here_API_KEY = "sFR8fGF9tv_-oQTgOPCZe4hcUH4kurAUCLc-zRmxcsA"  # Replace with your actual API key

# Coordinates for the start, via points, and end (ensure no spaces after commas)
start_coords = ("Start", "24.854194,46.712939")  # Start point (your location)
via_coords_1 = ("Via Point 1", "24.754636,46.635893")  # First via point
via_coords_2 = ("Via Point 2", "24.738056,46.710167")  # Second via point
end_coords = ("End", "24.678499,46.759968")    # End point



# List of all waypoints (excluding start)
waypoints = [via_coords_1, via_coords_2, end_coords]

# Helper function to get routes and congestion scores between two points
def get_route_and_congestion(origin, destination):
    # Directions API request URL
    origin_coords = origin[1]  # Extract the coordinates from the tuple
    destination_coords = destination[1]
    directions_url = f"https://router.hereapi.com/v8/routes?transportMode=car&origin={origin_coords}&destination={destination_coords}&return=summary,actions,polyline&alternatives=3&apiKey={here_API_KEY}"

    response = requests.get(directions_url)
    if response.status_code == 200:
        data = response.json()
        routes = data.get('routes', [])
        congestion_data = []

        for route in routes:
            polyline_data = route['sections'][0]['polyline']  # Encoded polyline data for the route
            route_coords = fp.decode(polyline_data)  # Decode the polyline into lat/lon pairs
            congestion_score = 0  # Initialize congestion score

            # Step 2: Get the traffic information along this route
            latitudes = [coord[0] for coord in route_coords]
            longitudes = [coord[1] for coord in route_coords]
            min_lat, max_lat = min(latitudes), max(latitudes)
            min_lng, max_lng = min(longitudes), max(longitudes)
            margin = 0.01  # Add margin to the bounding box
            bbox = f"{min_lng - margin},{min_lat - margin},{max_lng + margin},{max_lat + margin}"

            traffic_url = 'https://data.traffic.hereapi.com/v7/flow'
            traffic_params = {
                'apikey': here_API_KEY,
                'in': f"bbox:{bbox}",
                'units': 'metric',
                'locationReferencing': 'shape'
            }

            traffic_response = requests.get(traffic_url, params=traffic_params)
            if traffic_response.status_code == 200:
                traffic_data = traffic_response.json()

                # Calculate the congestion score by summing up jam factors
                for segment in traffic_data['results']:
                    current_flow = segment['currentFlow']
                    jam_factor = current_flow.get('jamFactor', 0)
                    congestion_score += jam_factor

            congestion_data.append((route, congestion_score))  # Store route and congestion score

        # Sort routes by congestion score (least congested first)
        congestion_data.sort(key=lambda x: x[1])
        return congestion_data[0]  # Return the route with the least congestion
    else:
        print(f"Error: {response.status_code}")
        return None

# Helper function to reorder waypoints based on least traffic
def reorder_waypoints(start, waypoints):
    ordered_journey = [start]  # Start the journey with the starting point
    current_location = start
    remaining_waypoints = waypoints[:]
    journey_legs = []

    while remaining_waypoints:
        best_next_leg = None
        least_congestion = float('inf')
        next_destination = None

        # Find the next destination with the least congestion
        for waypoint in remaining_waypoints:
            route, congestion = get_route_and_congestion(current_location, waypoint)
            print(f"Evaluating route from {current_location[0]} to {waypoint[0]}: Congestion Score = {congestion}")
            if congestion < least_congestion:
                least_congestion = congestion
                best_next_leg = route
                next_destination = waypoint

        print(f"Chosen next leg: {next_destination[0]} with Congestion Score = {least_congestion}\n")
        ordered_journey.append(next_destination)  # Add the next waypoint to the journey
        journey_legs.append(best_next_leg)  # Store the best route (leg) for mapping
        current_location = next_destination  # Move to the next destination
        remaining_waypoints.remove(next_destination)  # Remove the visited waypoint

    return ordered_journey, journey_legs

# Step 1: Reorder the journey based on traffic conditions
optimized_journey, journey_legs = reorder_waypoints(start_coords, waypoints)

# Print the optimized trip order with names and coordinates
print("Optimized Trip Order:")
for i, (name, coords) in enumerate(optimized_journey):
    print(f"{i + 1}. {name}: {coords}")

# Step 2: Initialize a folium map
mid_lat = (float(start_coords[1].split(",")[0]) + float(end_coords[1].split(",")[0])) / 2
mid_lon = (float(start_coords[1].split(",")[1]) + float(end_coords[1].split(",")[1])) / 2
route_map = folium.Map(location=[mid_lat, mid_lon], zoom_start=12)

# Add markers for the start, via points, and end
folium.Marker(
    location=[float(start_coords[1].split(",")[0]), float(start_coords[1].split(",")[1])],
    popup="Start",
    icon=folium.Icon(color="green")
).add_to(route_map)

# Add markers and polylines for the journey legs
for i, (best_route, destination) in enumerate(zip(journey_legs, optimized_journey[1:]), start=1):
    polyline_data = best_route['sections'][0]['polyline']  # Encoded polyline data for the route
    route_coords = fp.decode(polyline_data)  # Decode the polyline into lat/lon pairs

    # Add polyline for the leg
    folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.8).add_to(route_map)

    # Add numbered markers with styled square (white background, blue number)
    folium.Marker(
        location=[float(destination[1].split(",")[0]), float(destination[1].split(",")[1])],
        popup=f"{destination[0]}",
        icon=folium.DivIcon(html=f"""
            <div style="
                background-color: white;
                border: 2px solid blue;
                color: blue;
                font-size: 16px;
                font-weight: bold;
                text-align: center;
                width: 20px;
                height: 20px;">
                {i}
            </div>""")
    ).add_to(route_map)

# Step 3: Display the map in Google Colab or Jupyter
display(route_map)


Evaluating route from Start to Via Point 1: Congestion Score = 2127.5000000000036
Evaluating route from Start to Via Point 2: Congestion Score = 1610.5000000000023
Evaluating route from Start to End: Congestion Score = 3989.900000000008
Chosen next leg: Via Point 2 with Congestion Score = 1610.5000000000023

Evaluating route from Via Point 2 to Via Point 1: Congestion Score = 1590.9000000000003
Evaluating route from Via Point 2 to End: Congestion Score = 1896.1000000000024
Chosen next leg: Via Point 1 with Congestion Score = 1590.9000000000003

Evaluating route from Via Point 1 to End: Congestion Score = 4583.500000000003
Chosen next leg: End with Congestion Score = 4583.500000000003

Optimized Trip Order:
1. Start: 24.854194,46.712939
2. Via Point 2: 24.738056,46.710167
3. Via Point 1: 24.754636,46.635893
4. End: 24.678499,46.759968


# Display the Live Traffic and save it to CSV

In [ ]:
!pip install folium
!pip install flexpolyline

In [ ]:
import requests
import folium
import pandas as pd
from IPython.display import display

# Your HERE API key
api_key = 'sFR8fGF9tv_-oQTgOPCZe4hcUH4kurAUCLc-zRmxcsA'

# HERE Traffic API v7 endpoint
traffic_url = 'https://data.traffic.hereapi.com/v7/flow'

# Parameters for the API request
params = {
    'apikey': api_key,
    'in': 'bbox:46.627121,24.672364,46.788483,24.791412',  # Bounding box #46.627121,24.672364,46.788483,24.791412
    'units': 'metric',
    'locationReferencing': 'shape'
}

# Send a GET request to the HERE Traffic API v7
response = requests.get(traffic_url, params=params)

# Check if the request is successful
if response.status_code == 200:
    traffic_data = response.json()
    print("Traffic data fetched successfully.")

    # Initialize a map centered on Riyadh
    map_riyadh = folium.Map(location=[24.725923, 46.599927], zoom_start=14)

    # List to store the traffic data for saving to CSV
    traffic_info_list = []

    # Loop through the traffic data and extract coordinates, speed, and congestion details
    for segment in traffic_data['results']:
        location = segment['location']
        shape = location['shape']['links']  # Get the shape links

        for link in shape:
            points = link['points']  # Extract the points for each link

            # Create a list of (latitude, longitude) tuples for this link
            coordinates = [(point['lat'], point['lng']) for point in points]

            # Extract traffic data
            current_flow = segment['currentFlow']
            current_speed = current_flow.get('speed', 0)
            free_flow_speed = current_flow.get('freeFlow', 1)
            jam_factor = current_flow.get('jamFactor', 0)

            # Store each segment's data into a list for CSV
            traffic_info_list.append({
                'coordinates': coordinates,
                'current_speed_kmh': current_speed,
                'free_flow_speed_kmh': free_flow_speed,
                'jam_factor': jam_factor
            })

            # Determine traffic status and color
            if current_speed < free_flow_speed * 0.5:
                color = 'red'  # Heavy traffic
            elif current_speed < free_flow_speed * 0.8:
                color = 'orange'  # Moderate traffic
            else:
                color = 'green'  # Light traffic

            # Add the road segment to the map as a polyline
            folium.PolyLine(
                locations=coordinates,
                color=color,
                weight=5,
                opacity=0.8,
                popup=f"Current Speed: {current_speed} km/h, Jam Factor: {jam_factor}"
            ).add_to(map_riyadh)

    # Save the traffic data to a CSV file
    traffic_df = pd.DataFrame(traffic_info_list)
    traffic_df.to_csv("live_traffic_data.csv", index=False)
    print("Traffic data saved to 'live_traffic_data.csv'.")

    # Display the live traffic map
    display(map_riyadh)

else:
    print(f"Error fetching traffic data: {response.status_code}, {response.text}")

# Get the Road Network data and save it to CSV

In [ ]:
import requests
import pandas as pd
import flexpolyline

# Your HERE API key
api_key = 'sFR8fGF9tv_-oQTgOPCZe4hcUH4kurAUCLc-zRmxcsA'

# Bounding box (same as traffic)
bounding_box = '46.627121,24.672364,46.788483,24.791412' #46.627121,24.672364,46.788483,24.791412

# Fetch road network using Routing API within the bounding box (without specifying points C to D)
routing_url = f"https://router.hereapi.com/v8/routes?origin={bounding_box.split(',')[1]},{bounding_box.split(',')[0]}&destination={bounding_box.split(',')[3]},{bounding_box.split(',')[2]}&transportMode=car&return=polyline,summary&apiKey={api_key}"
routing_response = requests.get(routing_url)

# Check if Routing API request was successful
if routing_response.status_code == 200:
    routing_data = routing_response.json()
    print("Road network data fetched successfully.")

    # Initialize a list to store road network data
    road_data_list = []

    # Process routing data and extract road segments (polylines)
    for route in routing_data['routes']:
        for section in route['sections']:
            polyline = section['polyline']  # Encoded polyline representing the road segment
            summary = section['summary']    # Route summary (length, duration, etc.)

            # Decode the polyline to lat/lon points
            decoded_points = flexpolyline.decode(polyline)

            # Append each road segment's data
            road_data_list.append({
                'coordinates': decoded_points,
                'distance_m': summary['length'],
                'duration_s': summary['duration']
            })

    # Save road network data into CSV
    road_df = pd.DataFrame(road_data_list)
    road_df.to_csv("road_network_data.csv", index=False)
    print("Road network data saved to 'road_network_data.csv'.")

else:
    print(f"Error fetching road network: {routing_response.status_code}, {routing_response.text}")


Road network data fetched successfully.
Road network data saved to 'road_network_data.csv'.


# Going from (A) to (B) based on least traffic
#(Using the fetched data from the Traffic and Road Network files)

In [ ]:
import pandas as pd
import networkx as nx
import folium
from IPython.display import display
import ast
from geopy.distance import geodesic

# Coordinates for point C and point D (assigned here in Step 3)
start_coords = (24.703690, 46.758728)  # C
end_coords = (24.760355,46.692810)    # D

# Function to find the nearest node in the graph
def find_nearest_node(graph, target_coords):
    nearest_node = None
    min_distance = float('inf')

    for node in graph.nodes:
        distance = geodesic(node, target_coords).meters  # Use geodesic distance in meters
        if distance < min_distance:
            nearest_node = node
            min_distance = distance

    return nearest_node

# Step 3: Load saved traffic data and road network data
traffic_df = pd.read_csv("live_traffic_data.csv")
road_df = pd.read_csv("road_network_data.csv")

# Initialize a directed graph for road and traffic data
G = nx.DiGraph()

# Process the traffic data and build the graph based on traffic conditions
for idx, row in traffic_df.iterrows():
    coords_list = ast.literal_eval(str(row['coordinates']))  # Convert string to list of coordinates
    current_speed = row['current_speed_kmh']  # Speed in km/h

    # Add edges between each pair of coordinates with weights based on traffic
    for i in range(len(coords_list) - 1):
        point_a = tuple(coords_list[i])
        point_b = tuple(coords_list[i + 1])

        # Calculate travel time adjusted for traffic
        travel_time = 1 / current_speed if current_speed > 0 else float('inf')  # Handle impassable roads

        # Add directed edges to the graph with the weight based on travel time
        G.add_edge(point_a, point_b, weight=travel_time)

# Process the road network data and add it to the graph
for idx, row in road_df.iterrows():
    points = ast.literal_eval(str(row['coordinates']))  # Convert stringified list back to list

    # Add the road network to the graph
    for i in range(len(points) - 1):
        point_a = tuple(points[i])
        point_b = tuple(points[i + 1])

        # Add edges between the road points in the graph
        G.add_edge(point_a, point_b, weight=row['duration_s'])

# Find the nearest nodes to C and D in the graph
nearest_start_node = find_nearest_node(G, start_coords)
nearest_end_node = find_nearest_node(G, end_coords)

print(f"Nearest start node: {nearest_start_node}")
print(f"Nearest end node: {nearest_end_node}")

# Find the least traffic path from the nearest start node to the nearest end node
try:
    least_traffic_path = nx.dijkstra_path(G, source=nearest_start_node, target=nearest_end_node, weight='weight')
    print(f"Least traffic path from C to D: {least_traffic_path}")

    # Visualize the path on a map
    map_route = folium.Map(location=start_coords, zoom_start=14)

    # Add a marker for the starting point (C)
    folium.Marker(
        location=[nearest_start_node[0], nearest_start_node[1]],
        popup="Start (C)",
        icon=folium.Icon(color="green")
    ).add_to(map_route)

    # Add a marker for the ending point (D)
    folium.Marker(
        location=[nearest_end_node[0], nearest_end_node[1]],
        popup="End (D)",
        icon=folium.Icon(color="red")
    ).add_to(map_route)

    # Add the least traffic route to the map in blue
    folium.PolyLine(least_traffic_path, color="blue", weight=5, opacity=0.8).add_to(map_route)

    # Display the map
    display(map_route)

except nx.NetworkXNoPath:
    print(f"No path exists between {start_coords} and {end_coords}!")


Nearest start node: (24.70381, 46.75854)
Nearest end node: (24.76041, 46.69266)
Least traffic path from C to D: [(24.70381, 46.75854), (24.70282, 46.75911), (24.7023, 46.75942), (24.7016, 46.75984), (24.70143, 46.75991), (24.70078, 46.76016), (24.70061, 46.76023), (24.70041, 46.7603), (24.70004, 46.76045), (24.69987, 46.76053), (24.6985, 46.76106), (24.69836, 46.76111), (24.69785, 46.76131), (24.69676, 46.76175), (24.6962, 46.76198), (24.69599, 46.76205), (24.69569, 46.76218), (24.6951, 46.76241), (24.69472, 46.76256), (24.69309, 46.76323), (24.69272, 46.76339), (24.69182, 46.76374), (24.69172, 46.76373), (24.69165, 46.76369), (24.69161, 46.76361), (24.69154, 46.76346), (24.69146, 46.76334), (24.69108, 46.76209), (24.691, 46.76186), (24.69084, 46.76139), (24.69071, 46.76099), (24.69062, 46.76073), (24.69032, 46.7598), (24.69017, 46.75931), (24.68969, 46.75791), (24.68956, 46.75753), (24.68945, 46.75718), (24.68887, 46.75544), (24.6887, 46.75491), (24.68856, 46.75449), (24.68843, 46.754